In [ ]:
# %%writefile JsonNumpyEncodeDecode.py
from __future__ import absolute_import
from BashColors import C
from CV2_Utils_2 import *
import numpy as np
import cv2
import glob, json, os, pprint
pp=pprint.PrettyPrinter()
from os.path import *

contentPath=join(os.getcwd())
fileSavePath=join(contentPath, 'imagePathDictionary.json')

globList=glob.glob('**', recursive=True)
print(len(globList), 'files')
itemCount=0
imagePathDictionary={}
for item in globList:
    fullPath=abspath(item)
    if item.endswith('.png')or item.endswith('.jpg')or item.endswith('jpeg'):
        # thisItemCount=str(itemCount)
        imagePathDictionary.update([(itemCount, fullPath)])
        # print(itemCount, fullPath)
        itemCount+=1
print(len(imagePathDictionary), 'paths in imagePathDictionary')

class EncodeFromNumpy(json.JSONEncoder):
    """
    - Serializes python/Numpy objects via customizing json encoder.
    - **Usage**
        - `json.dumps(python_dict, cls=EncodeFromNumpy)` to get json string.
        - `json.dump(*args, cls=EncodeFromNumpy)` to create a file.json.
    """
    def default(self, obj):
        import numpy
        if isinstance(obj, numpy.ndarray):
            return {
                "_kind_": "ndarray",
                "_value_": obj.tolist()
            }
        if isinstance(obj, numpy.integer):
            return int(obj)
        elif isinstance(obj, numpy.floating):
            return float(obj)
        elif isinstance(obj,range):
            value = list(obj)
            return {
                "_kind_" : "range",
                "_value_" : [value[0],value[-1]+1]
            }
        return super(EncodeFromNumpy, self).default(obj)

class DecodeToNumpy(json.JSONDecoder):
    """
    - Deserilizes JSON object to Python/Numpy's objects.
    - **Usage**
        - `json.loads(json_string,cls=DecodeToNumpy)` from string, use `json.load()` for file.
    """
    def __init__(self, *args, **kwargs):
        json.JSONDecoder.__init__(self, object_hook=self.object_hook, *args, **kwargs)

    def object_hook(self, obj):
        import numpy
        if '_kind_' not in obj:
            return obj
        kind = obj['_kind_']
        if kind == 'ndarray':
            return numpy.array(obj['_value_'])
        elif kind == 'range':
            value = obj['_value_']
            return range(value[0],value[-1])
        return obj

efm=EncodeFromNumpy()
imageDict=efm.encode(imagePathDictionary)
with open(fileSavePath, 'w', encoding='utf-8') as f:
    json.dump(imageDict, f, ensure_ascii=False, indent=1)
# pp.pprint(imageDict)

dtn=DecodeToNumpy()
pathDictionary=dtn.decode(imageDict)
# pp.pprint(pathDictionary)


In [ ]:
 nameCount=0
def renameImage(image_path, base_Name, count=nameCount):
    ''' '''
    global nameCount
    nameCount += 1
    print(f'{C.BIRed}{basename(image_path)}{C.ColorOff}')
    splt = split(image_path)
    base_path = splt[0] + '/'
    print(base_path)
    new_path =  base_path + base_Name + str(count) + '.png'
    print(f'{C.BIGreen}{basename(new_path)}{C.ColorOff}')

keys=pathDictionary.keys()
for k in keys:
    thisPath=pathDictionary[k]
    if thisPath.__contains__('planets'):
        print(basename(thisPath))
        copy=np.copy(thisPath)
        copy=str(copy)
        img=cv2.imread(copy)
        # print(img.shape)
        if img.shape != (224, 224, 3):
            img=cv2.resize(img, (224, 224))
        # else: print('ok')
        savePath = renameImage(image_path=thisPath, base_Name='planet')
        # cv2.imwrite(thisPath, img)
        cvu.plotShowSingleImage(img, title1=img.shape)
        print()
        img=None

In [ ]:
class JsonNumpyUtils(EncodeFromNumpy, DecodeToNumpy):
    ''' '''
    from BashColors import C
    # import glob, json, os, pprint
    
    def __init__(self):
        '''init'''
        # self.pp = pprint.PrettyPrinter()
        self.created = self.getDate()
        self.contentPath=os.getcwd()
        # self.jsonFileSet={'q'}
        # self.jsonFileSet.remove('q')
        # self.jsu = JsonNumpyUtils
        
    def __iter__(self):
        '''iter'''
        return self
         # for key in self.some_sequence:
            # yield (key, 'Value for {}'.format(key))
             
    def getDate(self):
        '''returns formatted date string'''
        import time
        secs = time.time()
        secs = secs - (5 * 60 * 60)
        date=time.ctime(secs)
        return str(date)

In [ ]:
# import JsonNumpyUtils
from os.path import *
jnu=JsonNumpyUtils()

dirList=[]
dirListItems=dir(jnu)
for item in sorted(dirListItems):
    if item.__contains__('__'):
        pass
    else:
        dirList.append(item)
        # print(item)
        
print(dirList)